In [1]:
import numpy as np
import random
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import time

import sys
sys.path.append('/home/ksh-server/workspace/ICUFN')
from my_clustering.my_vmas.scenarios.navigation_clustering import Scenario

In [2]:
def list_pos(n_agents, n_targets):
    max_size = n_targets * 2
    pos = (random.randint(0, max_size), random.randint(0, max_size))
    
    agents_pos = [[random.randint(0, max_size), random.randint(0, max_size)] for _ in range(n_agents)]
    finished_pos = [[random.randint(0, max_size), random.randint(0, max_size)] for _ in range(n_agents)]
    targets_pos = [[random.randint(0, max_size), random.randint(0, max_size)] for _ in range(n_targets)]
    
    return agents_pos, finished_pos, targets_pos

In [3]:
def compute_euclidean_distance_matrix(locations):
    n = len(locations)
    matrix = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            if i != j:
                dx, dy = locations[i] - locations[j]
                matrix[i][j] = int(np.hypot(dx, dy))  # 소수점 줄이기 위해 스케일 업
    return matrix

In [4]:
def dist_matrix_for_ortools(targets_pos, agents_pos, finished_pos):
    all_pos = agents_pos + finished_pos + targets_pos

    locations = np.array(all_pos)
    matrix = compute_euclidean_distance_matrix(locations)
    matrix = matrix.tolist()
    
    return matrix

In [5]:
def create_data_model_for_ortools(distance_matrix, n_agents):
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = n_agents
    data["starts"] = [i for i in range(n_agents)]
    data["ends"] = [i+n_agents for i in range(n_agents)]
    return data

In [6]:
def get_routes_for_ortools(data, solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  routes_distance = []
  for vehicle_id in range(data["num_vehicles"]):
    route_distance = 0
    index = routing.Start(vehicle_id)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      previous_index = index
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
      route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
    routes.append(route)
    routes_distance.append(route_distance)
  return routes, routes_distance

In [7]:
def make_solutions_for_ortools(targets_pos, agents_pos, finished_pos, max_dist, n_agents):
    distance_matrix = dist_matrix_for_ortools(targets_pos, agents_pos, finished_pos)
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model_for_ortools(distance_matrix, n_agents)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        max_dist,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        routes, routes_distance = get_routes_for_ortools(data, solution, routing, manager)
        
    return data, manager, routing, solution, routes, routes_distance

        

In [8]:
def ortools_get_data(n_targets):
    n_targets = n_targets
    n_agents = n_targets // 30
    if n_targets % 30 > 0 :
        n_agents += 1
    max_dist = n_targets * 100

    start_time = time.time()
    agents_pos, finished_pos, targets_pos = list_pos(n_agents, n_targets)
    data, manager, routing, solution, routes, routes_distance = make_solutions_for_ortools(targets_pos, agents_pos, finished_pos, max_dist, n_agents)
    end_time = time.time()
    the_time = end_time - start_time

    return routes, routes_distance, the_time

In [9]:
import sys
sys.path.append('/home/ksh-server/workspace/ICUFN/my_clustering/my_vmas/pathfinding/data')
import csv

file = open('./data/ortools_data_output.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(['n_targets', 'n_agents', 'time', 'targets_per_agent'])

total = 5

for num in range(5, total+1, 5):
    n_targets = num
    n_agents = n_targets // 30
    if n_targets % 30 > 0 :
        n_agents += 1
        
    for _ in range(5):
        routes, routes_distance, the_time = ortools_get_data(n_targets)
        targets_per_agent = [len(route) - 2 for route in routes]
        writer.writerow([n_targets, n_agents, the_time, targets_per_agent])
file.close()